In [5]:
from warnings import filterwarnings

filterwarnings("ignore")

import pandas as pd
import numpy as np
from ta.volatility import average_true_range

from modules.indicators import zlema
from modules.backtester import long_only_backtester
from modules.data_fetcher import download_historical_data


In [6]:
symbol = "BTC-USDT"  # Best: BNB, BTC,

df_BTC = download_historical_data(symbol, "4hour")  # .loc["2021-11-20":]
df_BTC.dropna(inplace=True)
# df_BTC = df_BTC.asfreq("H").ffill()
print(df_BTC.shape)
df_BTC.head()


(11388, 7)


,Timestamp,Open,Close,High,Low,Amount,Volume
Date,,,,,,,
2018-01-01 01:00:00,1.514765e+09,12933.000001,13000.000000,13889.000000,12510.000000,5.534502,72706.427555
2018-01-01 05:00:00,1.514779e+09,13000.000000,13300.000000,13446.510264,12600.000001,5.281884,69154.117003
2018-01-01 09:00:00,1.514794e+09,13697.624103,13839.999981,13989.999997,13050.000000,2.927550,38829.908810
2018-01-01 13:00:00,1.514808e+09,13001.000000,13221.999990,13839.999982,12500.000000,5.069823,64820.202765
2018-01-01 17:00:00,1.514822e+09,13000.000001,13540.443994,13799.999998,13000.000000,4.926572,65319.109039


In [7]:
df_BTC["ZLEMA"] = zlema(df_BTC.Close)
df_BTC["HA_Close"] = (
    df_BTC["Open"] + df_BTC["Close"] + df_BTC["High"] + df_BTC["Low"]
) / 4
df_BTC["ATR"] = 2 * average_true_range(
    df_BTC["High"], df_BTC["Low"], df_BTC["Close"], 5
)

df_BTC.dropna(inplace=True)
df_BTC.head()

,Timestamp,Open,Close,High,Low,Amount,Volume,ZLEMA,HA_Close,ATR
Date,,,,,,,,,,
2018-01-03 01:00:00,1.514938e+09,14050.000000,14999.999999,15498.999000,13480.000002,11.932412,169643.663748,16999.999998,14507.249750,2739.248209
2018-01-03 05:00:00,1.514952e+09,14990.000000,15000.000001,15379.000000,14050.000000,4.984566,72691.487379,16844.230769,14854.750000,2722.998567
2018-01-03 09:00:00,1.514966e+09,15000.000000,14600.491246,15000.000001,13820.000000,6.950097,100914.862362,16311.314071,14605.122812,2650.398854
2018-01-03 13:00:00,1.514981e+09,14600.491248,14689.000001,15199.999999,13872.010002,7.585514,111221.309423,16267.895047,14590.375313,2651.515082
2018-01-03 17:00:00,1.514995e+09,14689.000002,14800.000000,15350.000000,14601.000000,8.579226,128591.815852,16219.580364,14860.000001,2420.812066


In [8]:
import plotly.graph_objects as go
from plotly.subplots import make_subplots

df_BTC_short = df_BTC.iloc[-200:]
fig = make_subplots(
    rows=1,
    cols=1,
    subplot_titles=("Historical price", "ZLEMA Distance"),
    shared_xaxes=True,
)

fig.add_trace(
    go.Candlestick(
        name="Historical price",
        x=df_BTC_short.index,
        open=df_BTC_short["Open"],
        high=df_BTC_short["High"],
        low=df_BTC_short["Low"],
        close=df_BTC_short["Close"],
    ),
    row=1,
    col=1,
)

fig.add_trace(
    go.Scatter(
        name="ZLEMA",
        x=df_BTC_short.index,
        y=df_BTC_short["ZLEMA"],
    ),
    row=1,
    col=1,
)

fig.add_trace(
    go.Scatter(
        name="ATR",
        x=df_BTC_short.index,
        y=df_BTC_short["ATR"],
    ),
    row=1,
    col=1,
)
fig.update_layout(
    xaxis_rangeslider_visible=False,
    showlegend=True,
    title_text="Historical price and ZLEMA",
)
fig.show()
